## Summarizing Chatbot w/f with Memory

In [17]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage,SystemMessage
from langgraph.graph.message import MessagesState, RemoveMessage
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)

class State(MessagesState):
    summary: str

In [19]:
def chat_model(state:State):
    summary = state.get("summary", "")

    if summary:
        system_message = f"Summary of the Conversation Earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    
    response = llm.invoke(messages)

    return {"messages": response}

In [20]:
def summarize_conversation(state:State):
    summary = state.get("summary", "")

    if summary:
        summary_message = (
            f"This Summary of the Conversation is to date: {summary} \n\n",
            "Extend the Summary by taking into account the new messages above:"
        )
    else:
        summary_message = "create a summary of the conversation above"
    
    messages = state["messages"] + [HumanMessage(content=summary_message)]

    response = llm.invoke(messages)

    deleted_messages = [RemoveMessage(msg.id) for msg in state["messages"][:-2]]

    return {
        "messages": deleted_messages,
        "summary": response.content
    }

In [22]:

def should_continue(state: State):
    """Return the Next Node to execute"""
    messages = state["messages"]

    if len(messages) > 6:
        return "summarize_conversation"
    
    return END


In [23]:
from IPython.display import display, Image
from langgraph.checkpoint.memory import MemorySaver

workflow = StateGraph(State)

workflow.add_node("conversation", chat_model)
workflow.add_node(summarize_conversation)

workflow.add_edge(START,"conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

ConnectTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Max retries exceeded with url: /img/JSV7aW5pdDogeydmbG93Y2hhcnQnOiB7J2N1cnZlJzogJ2xpbmVhcid9fX0lJQpncmFwaCBURDsKCV9fc3RhcnRfXyhbPHA+X19zdGFydF9fPC9wPl0pOjo6Zmlyc3QKCWNvbnZlcnNhdGlvbihjb252ZXJzYXRpb24pCglzdW1tYXJpemVfY29udmVyc2F0aW9uKHN1bW1hcml6ZV9jb252ZXJzYXRpb24pCglfX2VuZF9fKFs8cD5fX2VuZF9fPC9wPl0pOjo6bGFzdAoJX19zdGFydF9fIC0tPiBjb252ZXJzYXRpb247CglzdW1tYXJpemVfY29udmVyc2F0aW9uIC0tPiBfX2VuZF9fOwoJY29udmVyc2F0aW9uIC0uLT4gc3VtbWFyaXplX2NvbnZlcnNhdGlvbjsKCWNvbnZlcnNhdGlvbiAtLi0+IF9fZW5kX187CgljbGFzc0RlZiBkZWZhdWx0IGZpbGw6I2YyZjBmZixsaW5lLWhlaWdodDoxLjIKCWNsYXNzRGVmIGZpcnN0IGZpbGwtb3BhY2l0eTowCgljbGFzc0RlZiBsYXN0IGZpbGw6I2JmYjZmYwo=?bgColor=!white (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000250F3C47E90>, 'Connection to mermaid.ink timed out. (connect timeout=10)'))

In [24]:
config = {"configurable" : {"thread_id" : "1"}}

input_message = HumanMessage(content="Hi, this is Hijabie")
out_put = graph.invoke({"messages" : [input_message]}, config)
for m in out_put["messages"][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="What is my Name?")
out_put = graph.invoke({"messages" : [input_message]}, config)
for m in out_put["messages"][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="tell me about white Wolf?")
out_put = graph.invoke({"messages" : [input_message]}, config)
for m in out_put["messages"][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

Hello Hijabie! How can I assist you today?
================================== Ai Message ==================================

Your name is Hijabie. How can I help you today?
================================== Ai Message ==================================

The term "white wolf" can refer to a few different concepts, depending on the context. Here are a few interpretations:

1. **Animal**: The white wolf is often associated with the Arctic wolf (Canis lupus arctos), which has a white coat that helps it blend into its snowy environment. These wolves are found in the Arctic regions of North America and Greenland. They are known for their social behavior, living in packs, and their adaptability to harsh climates.

2. **Symbolism**: In various cultures, the white wolf symbolizes purity, loyalty, and a strong connection to nature. It is often seen as a spiritual guide or a protector in Native American folklore. T

In [27]:
graph.get_state(config).values.get("summary","")

''

In [28]:
input_message = HumanMessage(content="History of Wolf in Turkish Empire?")
out_put = graph.invoke({"messages" : [input_message]}, config)
for m in out_put["messages"][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

Wolves hold a significant place in the mythology and history of the Turkish people, particularly during the time of the Turkish Empire and its predecessors. Here are some key points regarding the role and symbolism of wolves in Turkish history:

### 1. **Mythological Significance**:
   - **Turkic Mythology**: In ancient Turkic mythology, the wolf is often seen as a sacred animal. The legend of the "Gray Wolf" (Kurt) is particularly important. According to some myths, the Turks trace their lineage back to a she-wolf who nurtured a boy named Asena. This story symbolizes strength, resilience, and a connection to nature.
   - **Cultural Symbol**: The wolf is often associated with bravery, loyalty, and the warrior spirit. It is a symbol of the Turkic identity and has been used in various emblems and flags.

### 2. **Historical Context**:
   - **Gokturk Khaganate**: The Gokturks, one of the first Turkic states 

In [29]:
graph.get_state(config).values.get("summary","")

'In our conversation, you first introduced yourself as Hijabie and asked about the "white wolf." I provided information on various interpretations of the term, including its significance as an animal (specifically the Arctic wolf), its symbolism in different cultures, its representation in fiction and media (notably in "The Witcher"), and its association with White Wolf Publishing, known for tabletop role-playing games.\n\nYou then inquired about the history of wolves in the Turkish Empire. I explained that wolves hold a significant place in Turkic mythology, particularly as a symbol of strength and resilience. I mentioned the legend of the Gray Wolf and its importance to Turkic identity, as well as the historical context of wolves in the Gokturk Khaganate and their use as military symbols. Additionally, I noted the cultural influence of wolves in folklore and contemporary references, including their association with nationalism in modern Turkey.'